In [ ]:
import requests  
import pandas as pd  
from bs4 import BeautifulSoup 
import matplotlib.pyplot as plt 
import seaborn as sns 
from dotenv import load_dotenv
import os

load_dotenv()

sns.set_style("whitegrid")

Now lets get info from the The Movie DB API

In [3]:
API_KEY = os.getenv('API_KEY')
API_BASE_URL = 'https://api.themoviedb.org/3'


movie_data_list = []
for page in range(1, 11):
    url = f"{API_BASE_URL}/discover/movie?api_key={API_KEY}&primary_release_year=2025&sort_by=popularity.desc&page={page}"
    
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        movie_data_list.extend(data['results'])
    else:
        print(f"Error fetching page {page}: {response.status_code}")
        break

print(movie_data_list)

[{'adult': False, 'backdrop_path': '/7QirCB1o80NEFpQGlQRZerZbQEp.jpg', 'genre_ids': [10749, 18], 'id': 1156594, 'original_language': 'es', 'original_title': 'Culpa nuestra', 'overview': "Jenna and Lion's wedding brings about the long-awaited reunion between Noah and Nick after their breakup. Nick's inability to forgive Noah stands as an insurmountable barrier. He, heir to his grandfather's businesses, and she, starting her professional life, resist fueling a flame that's still alive. But now that their paths have crossed again, will love be stronger than resentment?", 'popularity': 1201.3045, 'poster_path': '/yzqHt4m1SeY9FbPrfZ0C2Hi9x1s.jpg', 'release_date': '2025-10-15', 'title': 'Our Fault', 'video': False, 'vote_average': 8.179, 'vote_count': 151}, {'adult': False, 'backdrop_path': '/2yo2k8CmgZxfBQYAZ8TnHIYzEme.jpg', 'genre_ids': [28, 80, 53], 'id': 1072699, 'original_language': 'pl', 'original_title': 'Druga Furioza', 'overview': 'In the wake of murder, new Furioza leader Golden cl

In [6]:
tmdb_df = pd.DataFrame(movie_data_list)
tmdb_df = tmdb_df[['title', 'vote_average', 'release_date', 'popularity']]

print(f"Successfully fetched data for {len(tmdb_df)} movies from TMDb.")
tmdb_df.head()

Successfully fetched data for 200 movies from TMDb.


,title,vote_average,release_date,popularity
0,Our Fault,8.179,2025-10-15,1201.3045
1,Inside Furioza,6.545,2025-10-14,366.6376
2,The Conjuring: Last Rites,6.957,2025-09-03,348.3929
3,War of the Worlds,4.400,2025-07-29,286.9612
4,Demon Slayer: Kimetsu no Yaiba Infinity Castle,7.806,2025-07-18,275.9199
